In [1]:
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpMaximize, LpStatus, PULP_CBC_CMD
from datetime import datetime, timedelta
import pandas as pd

In [2]:
# extract data
courses = pd.read_csv('courses.csv')
venues = pd.read_csv('venues.csv')

# drop unavailable venues
available_venues = venues[venues['isAvailable'] == 'yes']

# separate MW and TR courses
MW_courses = courses[courses['Days'] == 'MW']
TR_courses = courses[courses['Days'] == 'TR']

# reset index
MW_courses.reset_index(drop=True, inplace=True)
TR_courses.reset_index(drop=True, inplace=True)

In [3]:
# weights for the objective function

# max utilization ratio
def w1(i, j):
    if MW_courses['Enrollment'][i] <= available_venues['Seating Capacity'][j]:
        return MW_courses['Enrollment'][i] / available_venues['Seating Capacity'][j]
    return 0

def weighted_product(i, j, x):
    return w1(i, j)

In [ ]:
problem = LpProblem("Class-Scheduling", LpMaximize)

# each course can be scheduled in one of the available venues
x = LpVariable.dicts("x", [(i, j) for i in range(MW_courses.shape[0]) for j in range(len(available_venues))], 0, 1, LpBinary)

# objective function
problem += lpSum(x[i, j] * weighted_product(i, j, x) for i in range(MW_courses.shape[0]) for j in range(len(available_venues)))

# Constraint: Each course is scheduled in exactly one venue
for i in range(MW_courses.shape[0]):
    problem += lpSum(x[i, j] for j in range(len(available_venues))) == 1

# Constraint: Assigned venue has enough seating capacity
for i in range(MW_courses.shape[0]):
    for j in range(len(available_venues)):
        if MW_courses['Enrollment'][i] > available_venues['Seating Capacity'][j]:
            problem += x[i, j] == 0

# Constraint: No overlapping courses in the same venue
for j in range(len(available_venues)):
    for i in range(MW_courses.shape[0]):
        overlaps = [x[k, j] for k in range(MW_courses.shape[0]) if i != k and (MW_courses['Start Time'][i] < MW_courses['End Time'][k] and MW_courses['End Time'][i] > MW_courses['Start Time'][k])]
        if len(overlaps) > 0:
            problem += x[i, j] + lpSum(overlaps) <= 1

In [15]:
# solve the problem
problem.solve(PULP_CBC_CMD())

-1

In [6]:
# # print the results
# print("Course Code, Enrollment, Start Time, End Time, Venue, Seating Capacity")
# for i in range(MW_courses.shape[0]):
#     for j in range(len(available_venues)):
#         if x[i, j].value() == 1:
#             print(f"{MW_courses['Course Code'][i]}, {MW_courses['Enrollment'][i]}, {MW_courses['Start Time'][i]}, {MW_courses['End Time'][i]}, {available_venues['Class Venues'][j]}, {available_venues['Seating Capacity'][j]}")

# # # classes scheduled per venue
# print("\nVenue, Classes Scheduled")
# for j in range(len(available_venues)):
#     print(f"{available_venues['Class Venues'][j]}, {sum(x[i, j].value() for i in range(MW_courses.shape[0]))}")